In [49]:
import data
import utils
import json
import torch
import torchvision
import numpy as np
import sklearn.metrics as skm
import torchvision.models.segmentation.deeplabv3 as dlv3
from torch.utils.data import DataLoader
from torch.utils.data import Subset
from tqdm import tqdm

## Test-related functions

In [50]:
def eval(model, test_dataloader, tgt_num_classes, tgt_ignore_label=None):
    model.eval()
    average_miou = 0
    average_ious = np.zeros((tgt_num_classes,))
    with torch.no_grad():
        for (inputs, targets) in tqdm(test_dataloader, desc='Batch progress'):
            ipts = torch.autograd.Variable(inputs).cuda()
            tgts = torch.autograd.Variable(targets).cuda()
            
            preds = model(ipts)['out']
            preds = torch.argmax(preds.cpu(), dim=1)
            tgts = torch.squeeze(targets, dim=1)

            miou, ious = utils.measure_performance(predictions=preds, targets=tgts, num_classes=tgt_num_classes, ignore_label=tgt_ignore_label)
            average_ious += ious
            average_miou += miou
            
        average_ious /= len(test_dataloader)
        average_miou /= len(test_dataloader)

    return {'average_miou': np.round(average_miou, 3), 'average_ious': np.round(average_ious, 3)}

## Test configuration loading

In [51]:
import pprint
args = {}
with open('G:/My Drive/Master IVA/Master Thesis/Configs/eval-config-1.json') as json_file:
    args = json.load(json_file)
pprint.pprint(args)

{'data_source': 'real',
 'ignore_label': 19,
 'image_height': 256,
 'image_width': 256,
 'model_load_path': 'G:/My Drive/Master IVA/Master '
                    'Thesis/Models/20230322_214215/deeplabv3_model.pt',
 'test_batch_size': 4,
 'test_subset_size': 250}


## Data loading and preprocessing

In [52]:
test_dataset = data.HybridDataset(
        root_path=f'G:/My Drive/Master IVA/Master Thesis/Datasets/{args["data_source"]}/test',
        input_dir='rgb',
        target_dir='semantic_segmentation_mapped',
        ipt_transform=torchvision.transforms.Compose([torchvision.transforms.Resize((args["image_height"], args["image_width"]))]),
        tgt_transform=torchvision.transforms.Compose([torchvision.transforms.Resize((args["image_height"], args["image_width"]))]),                        
        type=args['data_source'],
        labels_mapping=None)
test_dataset = Subset(test_dataset, indices=range(250, 250 + args['test_subset_size'], 1))
test_dataloader = DataLoader(
    dataset=test_dataset,
    batch_size=args['test_batch_size'],
    shuffle=False
)

In [53]:
test_pixel_sum = torch.tensor([0.0, 0.0, 0.0])
test_pixel_sum_sq = torch.tensor([0.0, 0.0, 0.0])

for test_inputs, _ in tqdm(test_dataloader, desc='Test dataset norm. params. comp. progress'):
    test_pixel_sum += test_inputs.sum(axis = [0, 2, 3])
    test_pixel_sum_sq += (test_inputs ** 2).sum(axis = [0, 2, 3])

test_pixel_count = args["test_subset_size"] * args["image_width"] * args["image_height"]
test_mean = test_pixel_sum / test_pixel_count
test_variance = (test_pixel_sum_sq / test_pixel_count) - (test_mean ** 2)
test_std = torch.sqrt(test_variance)

print(f'Test dataset \'mean\':{test_mean}')
print(f'Test dataset \'std deviation\':{test_std}')

Test dataset norm. params. comp. progress: 100%|██████████| 63/63 [00:33<00:00,  1.87it/s]

Test dataset 'mean':tensor([0.2604, 0.3050, 0.2668])
Test dataset 'std deviation':tensor([0.1741, 0.1780, 0.1736])


In [54]:
test_dataset = data.HybridDataset(
        root_path=f'G:/My Drive/Master IVA/Master Thesis/Datasets/{args["data_source"]}/test',
        input_dir='rgb',
        target_dir='semantic_segmentation_mapped',
        ipt_transform=torchvision.transforms.Compose([torchvision.transforms.Resize((args["image_height"], args["image_width"])),
                                                      torchvision.transforms.Normalize(mean=test_mean, std=test_std)]),
        tgt_transform=torchvision.transforms.Compose([torchvision.transforms.Resize((args["image_height"], args["image_width"]))]),                        
        type=args['data_source'],
        labels_mapping=None)
test_dataset = Subset(test_dataset, indices=range(250, 250 + args['test_subset_size'], 1))
test_dataloader = DataLoader(
    dataset=test_dataset,
    batch_size=args['test_batch_size'],
    shuffle=False
)

## Model under test preparation

In [55]:
# File parameters
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = dlv3.deeplabv3_resnet50(pretrained=False, progress=True, num_classes=len(data.SemanticLabelMapper.ID_TO_STRING['common'].keys()) - 1)
model.load_state_dict(torch.load(args['model_load_path']))
model.to(device)
print('Loaded model to device.')

Loaded model to device.


In [56]:
test_metrics = eval(model=model, 
                    test_dataloader=test_dataloader, 
                    tgt_num_classes=len(data.SemanticLabelMapper.ID_TO_STRING['common'].keys()),
                    tgt_ignore_label=args["ignore_label"])
test_metrics['average_ious'] = {k:v for k,v in zip(data.SemanticLabelMapper.ID_TO_STRING['common'].values(),
                                                   test_metrics['average_ious'])}
print(test_metrics)

Batch progress: 100%|██████████| 63/63 [01:03<00:00,  1.01s/it]

{'average_miou': 0.3, 'average_ious': {'road': 0.857, 'sidewalk': 0.618, 'building': 0.763, 'wall': 0.109, 'fence': 0.049, 'pole': 0.068, 'trafficlight': 0.054, 'trafficsign': 0.275, 'vegetation': 0.786, 'terrain': 0.265, 'sky': 0.398, 'pedestrian': 0.192, 'rider': 0.081, 'car': 0.742, 'truck': 0.039, 'bus': 0.108, 'train': 0.012, 'motorcycle': 0.007, 'bicycle': 0.272, 'any': 0.0}}
